In [ ]:

!pip install kaggle
!pip install librosa


import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import os
import glob
import json
import random
from scipy.io.wavfile import write

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
from google.colab import files
files.upload()
!kaggle datasets download -d tariqblecher/ravdess-8k
!kaggle datasets download -d tariqblecher/urban-sound-8k
!unzip \*.zip
!ls

ravdess_files = glob.glob('/content/ravdess-emotional-speech-audio/**/*.wav', recursive=True)
urban_files = glob.glob('/content/urbansound8k/**/*.wav', recursive=True)

white_noise1_path = "/content/01-White-Noise-10min.wav"
white_noise2_path = "/content/01-White-Noise-10min.wav"

white_noise1, _ = librosa.load(white_noise1_path)
white_noise2, _ = librosa.load(white_noise2_path)

ipd.Audio(white_noise1_path)
ipd.Audio(white_noise2_path)

white_noise1 = white_noise1[:88200]
white_noise2 = white_noise2[:88200]


print(len(ravdess_files))
print(len(urban_files))
print(len(white_noise1))


data = { "corr_mel_spectrogram": [], "clean_mel_spectrogram": []}

count = 1
mixed_audio_files = {}

for path in ravdess_files:
    if count % 10 == 0:
        print(count)
    count += 1

    randomint = []
    audio, sr = librosa.load(path)

    for i in range(10):
        x = random.randint(0, len(urban_files) - 1)
        randomint.append(x)

    for j in randomint:
        noise, _ = librosa.load(urban_files[j])

        if len(noise) <= 88200:
            noise = np.pad(noise, (0, 88200 - len(noise)))
        if len(noise) > 88200:
            noise = noise[:88200]

        if len(noise) <= len(audio):
            audio = audio[:len(noise)]
        else:
            audio = np.pad(audio, (0, len(noise) - len(audio)), "constant")

        reduced_noise = noise * 0.05
        white_noise = white_noise1 * 0.01 + white_noise2 * 0.01

        corr_audio = audio + reduced_noise + white_noise

        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=2048, hop_length=512, n_mels=10)
        corr_mel_spectrogram = librosa.feature.melspectrogram(y=corr_audio, sr=sr, n_fft=2048, hop_length=512, n_mels=10)

        log_mel_spectrogram_audio = librosa.power_to_db(mel_spectrogram)
        corr_log_mel_spectrogram_audio = librosa.power_to_db(corr_mel_spectrogram)

        data["clean_mel_spectrogram"].append(log_mel_spectrogram_audio.tolist())
        data["corr_mel_spectrogram"].append(corr_log_mel_spectrogram_audio.tolist())



        base_filename = os.path.basename(path).replace('.wav', f'_mixed_{j}.wav')
        mixed_audio_path = os.path.join('/content/mixed_audios', base_filename)


        os.makedirs(os.path.dirname(mixed_audio_path), exist_ok=True)
        write(mixed_audio_path, sr, corr_audio.astype(np.float32))


        mixed_audio_files[base_filename] = mixed_audio_path
with open('data.json', 'w') as f:
    json.dump(data, f)
with open('mixed_audio_files.json', 'w') as f:
    json.dump(mixed_audio_files, f)

print("Processing complete and data saved!")

def play_mixed_audio(filename):
    if filename in mixed_audio_files:
        path = mixed_audio_files[filename]
        return ipd.Audio(path)
    else:
        print(f"File {filename} not found in the dictionary.")
        return None
#play_mixed_audio(list(mixed_audio_files.keys())[0])  # Play the first mixed audio file